### MID 7
### Taking ant1.3 as training data.
### ant1.4 as testing data.
### Min max scaling done to few columns: ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
### Oversampling and smote methods used to increase size of training data.
###  Deep neural network used, model type is 3 as per the BTP documentation spreadsheet.
### np.rint() used on predictions

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
import tensorflow as tf
import csv
from sklearn.decomposition import TruncatedSVD

In [2]:
# Loading data. 
train_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv"
test_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"
train_data_name = "ant-1.3"
test_data_name = "ant-1.4"
ant_1_3 = pd.read_csv(train_data_path)
ant_1_4 = pd.read_csv(test_data_path)
files = ["/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv", "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"]
combined_data = pd.concat(map(pd.read_csv, files))

In [3]:
# Applying Min Max Scaling.
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
combined_data[cols_to_norm] = MinMaxScaler().fit_transform(combined_data[cols_to_norm])

In [6]:
len(cols_to_norm)

20

In [4]:
combined_data

name  version                                             name.1  \
0    ant      1.3            org.apache.tools.ant.taskdefs.ExecuteOn   
1    ant      1.3                 org.apache.tools.ant.DefaultLogger   
2    ant      1.3     org.apache.tools.ant.taskdefs.TaskOutputStream   
3    ant      1.3                  org.apache.tools.ant.taskdefs.Cvs   
4    ant      1.3             org.apache.tools.ant.taskdefs.Copyfile   
..   ...      ...                                                ...   
173  ant      1.4                   org.apache.tools.ant.TaskAdapter   
174  ant      1.4  org.apache.tools.ant.taskdefs.rmic.DefaultRmic...   
175  ant      1.4           org.apache.tools.ant.IntrospectionHelper   
176  ant      1.4  org.apache.tools.ant.taskdefs.compilers.Defaul...   
177  ant      1.4                org.apache.tools.ant.NoBannerLogger   

          wmc  dit    noc       cbo       rfc      lcom        ca  ...  \
0    0.142857  0.6  0.050  0.102941  0.214286  0.011856  0.014815  ...   
1    0.181818  0.0  0.025  0.058824  0.163265  0.020033  0.029630  ...   
2    0.038961  0.2  0.000  0.007353  0.045918  0.000000  0.000000  ...   
3    0.155844  0.4  0.000  0.088235  0.188776  0.013083  0.000000  ...   
4    0.077922  0.4  0.000  0.029412  0.107143  0.000409  0.000000  ...   
..        ...  ...    ...       ...       ...       ...       ...  ...   
173  0.064935  0.4  0.000  0.051471  0.096939  0.001635  0.029630  ...   
174  0.220779  0.0  0.075  0.110294  0.336735  0.036795  0.029630  ...   
175  0.298701  0.0  0.000  0.227941  0.418367  0.082175  0.162963  ...   
176  0.142857  0.0  0.200  0.161765  0.438776  0.006132  0.059259  ...   
177  0.051948  0.2  0.000  0.022059  0.081633  0.000000  0.000000  ...   

          dam       moa       mfa       cam    ic       cbm       amc  \
0    1.000000  0.111111  0.885057  0.232323  0.75  0.363636  0.165951   
1    1.000000  0.000000  0.000000  0.307692  0.00  0.000000  0.080979   
2    1.000000  0.111111  0.714286  0.666667  0.25  0.090909  0.083267   
3    1.000000  0.111111  0.770833  0.458333  0.00  0.000000  0.115693   
4    1.000000  0.000000  0.880952  0.416667  0.50  0.181818  0.100881   
..        ...       ...       ...       ...   ...       ...       ...   
173  0.000000  0.000000  0.902439  0.400000  0.25  0.090909  0.138351   
174  1.000000  0.222222  0.000000  0.197917  0.00  0.000000  0.172373   
175  0.444444  0.000000  0.000000  0.318182  0.00  0.000000  0.219724   
176  1.000000  0.777778  0.000000  0.266667  0.00  0.000000  0.373390   
177  1.000000  0.000000  0.842105  0.875000  0.25  0.090909  0.064852   

       max_cc    avg_cc  bug  
0    0.085714  0.209085    0  
1    0.171429  0.269903    2  
2    0.028571  0.109529    0  
3    0.085714  0.232742    0  
4    0.028571  0.136898    0  
..        ...       ...  ...  
173  0.028571  0.131428    0  
174  0.400000  0.396221    0  
175  0.685714  0.378561    0  
176  0.628571  0.761689    1  
177  0.142857  0.287498    0  

[303 rows x 24 columns]

In [5]:
ant_1_3_scaled = combined_data[combined_data['version'] == 1.3] 
ant_1_4_scaled = combined_data[combined_data['version'] == 1.4] 

In [6]:
# Getting training and test data.
X_train = ant_1_3_scaled[cols_to_norm]
X_train = np.array(X_train)
Y_train = ant_1_3['bug']

X_test = ant_1_4_scaled[cols_to_norm]
X_test = np.array(X_test)
Y_test = ant_1_4['bug']

In [7]:
# Applying over sampling and SMOTE to training data for augmentation.
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)

smt = SMOTE()
X_train, Y_train = smt.fit_resample(X_train, Y_train)

train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [8]:
train_x

array([[[0.14285714, 0.6       , 0.05      , ..., 0.16595094,
         0.08571429, 0.20908494]],

       [[0.18181818, 0.        , 0.025     , ..., 0.08097907,
         0.17142857, 0.26990307]],

       [[0.03896104, 0.2       , 0.        , ..., 0.08326661,
         0.02857143, 0.1095285 ]],

       ...,

       [[0.22077922, 0.        , 0.        , ..., 0.13535534,
         0.05714286, 0.16428454]],

       [[0.22077922, 0.        , 0.        , ..., 0.13535534,
         0.05714286, 0.16428454]],

       [[0.07792208, 0.4       , 0.        , ..., 0.15132106,
         0.05714286, 0.16428454]]])

In [9]:
Y_train

0      0
1      2
2      0
3      0
4      0
      ..
415    3
416    3
417    3
418    3
419    3
Name: bug, Length: 420, dtype: int64

In [10]:
train_y = Y_train.to_numpy()
train_y

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 3, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [11]:
test_x

array([[[0.18181818, 0.4       , 0.        , ..., 0.12421366,
         0.11428571, 0.25815673]],

       [[0.06493506, 0.4       , 0.        , ..., 0.25748599,
         0.02857143, 0.13142763]],

       [[0.16883117, 0.        , 0.025     , ..., 0.03473548,
         0.02857143, 0.12636767]],

       ...,

       [[0.2987013 , 0.        , 0.        , ..., 0.2197236 ,
         0.68571429, 0.37856087]],

       [[0.14285714, 0.        , 0.2       , ..., 0.37338962,
         0.62857143, 0.76168885]],

       [[0.05194805, 0.2       , 0.        , ..., 0.06485188,
         0.14285714, 0.28749795]]])

In [12]:
Y_test

0      0
1      0
2      0
3      0
4      0
      ..
173    0
174    0
175    0
176    1
177    0
Name: bug, Length: 178, dtype: int64

In [13]:
test_y = Y_test.to_numpy()
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0])

In [16]:
# Designing and initializing the model.
model = Sequential()
model.add(Dense(15, activation = 'relu', input_dim = 20))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(5, activation = 'relu'))
# Adding the output layer
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                315       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 128       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 494
Trainable params: 494
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100, batch_size = 128)

Epoch 1/100
4/4 [==============================] - 0s 821us/step - loss: 3.0476 - mse: 3.0476 - mae: 1.4121 - root_mean_squared_error: 1.7457 - mean_squared_logarithmic_error: 0.6974
Epoch 2/100
4/4 [==============================] - 0s 867us/step - loss: 2.8742 - mse: 2.8742 - mae: 1.3765 - root_mean_squared_error: 1.6953 - mean_squared_logarithmic_error: 0.6278
Epoch 3/100
4/4 [==============================] - 0s 1ms/step - loss: 2.7013 - mse: 2.7013 - mae: 1.3407 - root_mean_squared_error: 1.6436 - mean_squared_logarithmic_error: 0.5645
Epoch 4/100
4/4 [==============================] - 0s 1ms/step - loss: 2.5255 - mse: 2.5255 - mae: 1.3020 - root_mean_squared_error: 1.5892 - mean_squared_logarithmic_error: 0.5062
Epoch 5/100
4/4 [==============================] - 0s 1ms/step - loss: 2.3616 - mse: 2.3616 - mae: 1.2644 - root_mean_squared_error: 1.5368 - mean_squared_logarithmic_error: 0.4570
Epoch 6/100
4/4 [==============================] - 0s 1ms/step - loss: 2.2046 - mse: 2.2046

4/4 [==============================] - 0s 1ms/step - loss: 0.6184 - mse: 0.6184 - mae: 0.6412 - root_mean_squared_error: 0.7864 - mean_squared_logarithmic_error: 0.1476
Epoch 88/100
4/4 [==============================] - 0s 700us/step - loss: 0.6143 - mse: 0.6143 - mae: 0.6381 - root_mean_squared_error: 0.7838 - mean_squared_logarithmic_error: 0.1475
Epoch 89/100
4/4 [==============================] - 0s 840us/step - loss: 0.6093 - mse: 0.6093 - mae: 0.6349 - root_mean_squared_error: 0.7806 - mean_squared_logarithmic_error: 0.1461
Epoch 90/100
4/4 [==============================] - 0s 955us/step - loss: 0.6047 - mse: 0.6047 - mae: 0.6317 - root_mean_squared_error: 0.7776 - mean_squared_logarithmic_error: 0.1450
Epoch 91/100
4/4 [==============================] - 0s 661us/step - loss: 0.5990 - mse: 0.5990 - mae: 0.6277 - root_mean_squared_error: 0.7739 - mean_squared_logarithmic_error: 0.1437
Epoch 92/100
4/4 [==============================] - 0s 1ms/step - loss: 0.5946 - mse: 0.5946 - 

In [19]:
# Getting the predictions.
predictions_y = model.predict(test_x)
predictions_y

array([[[1.1089137 ]],

       [[1.8231691 ]],

       [[0.38210663]],

       [[1.532444  ]],

       [[1.6698124 ]],

       [[0.9863473 ]],

       [[1.3408384 ]],

       [[0.08152281]],

       [[0.3783476 ]],

       [[2.577356  ]],

       [[1.2922081 ]],

       [[2.3811834 ]],

       [[0.42386243]],

       [[1.7105166 ]],

       [[1.3794776 ]],

       [[0.08253507]],

       [[2.7667844 ]],

       [[0.17231804]],

       [[0.0986888 ]],

       [[0.53403914]],

       [[0.5576303 ]],

       [[0.47758737]],

       [[0.6695603 ]],

       [[1.4009397 ]],

       [[2.0162957 ]],

       [[1.2844194 ]],

       [[2.0242908 ]],

       [[0.80374885]],

       [[1.0040702 ]],

       [[0.08186112]],

       [[0.08664873]],

       [[0.2912376 ]],

       [[0.15667772]],

       [[1.1408505 ]],

       [[0.08955269]],

       [[0.08482862]],

       [[1.0432185 ]],

       [[0.098057  ]],

       [[0.5944849 ]],

       [[0.12650415]],

       [[1.7425222 ]],

       [[0.08409

In [20]:
# Rounding off the predictions to nearest
#integer as count of bugs is an integer.
predictions_y_round = np.rint(predictions_y)
predictions_y_round 

array([[[1.]],

       [[2.]],

       [[0.]],

       [[2.]],

       [[2.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[3.]],

       [[1.]],

       [[2.]],

       [[0.]],

       [[2.]],

       [[1.]],

       [[0.]],

       [[3.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[2.]],

       [[1.]],

       [[2.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[2.]],

       [[0.]],

       [[0.]],

       [[2.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[2.]],

       [[2.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[0.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [

In [21]:
# Viewing the number of predictions.
predictions_y_round.shape

(178, 1, 1)

In [22]:
# Getting the sum of all the predictions obtained to used while obtaining FPA
s = 0
for  t in range(predictions_y_round.shape[0]):
    s+=predictions_y_round[t]
s

array([[141.]], dtype=float32)

In [23]:
# Obtaining the value of FPA metric for the model
Fpa = 0
for  t in range(predictions_y_round.shape[0]):
        x = 0
        for j in range( predictions_y_round.shape[0]-t+1, predictions_y_round.shape[0]):
               x = x + predictions_y_round[j]
        
        x = (x/s)/predictions_y_round.shape[0]
        Fpa = Fpa + x
Fpa

array([[0.47545615]], dtype=float32)

In [24]:
# Obtaining the value of CLC metric for the model
previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s

CLC = 0
for i in range(predictions_y_round.shape[0]):
    if(i==0):
        CLC += 0 + previous_obtained
    else:
        additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s
        CLC += 2*previous_obtained + additional
        previous_obtained += additional
        
CLC/=(2*predictions_y_round.shape[0])
CLC

array([[0.48384327]], dtype=float32)

In [25]:
# Getting direct metric results using the metrics given to model.
score = model.evaluate(test_x, test_y)
print(history.history.keys())
model.test_on_batch(test_x, test_y)
model.metrics_names
#print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))


6/6 [==============================] - 0s 711us/step - loss: 1.1668 - mse: 1.1668 - mae: 0.8208 - root_mean_squared_error: 1.0802 - mean_squared_logarithmic_error: 0.3858
dict_keys(['loss', 'mse', 'mae', 'root_mean_squared_error', 'mean_squared_logarithmic_error'])


['loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error']

In [26]:
# Saving the model
model_id = 7
path_to_save = '/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model3/ant1.3_ant1.4_model3_1'
model.save(path_to_save)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model3/ant1.3_ant1.4_model3_1/assets


In [27]:
# Writing the results to csv file.
heading = ['model_id', 'train_data_name', 'test_data_name'] + model.metrics_names + ['fpa', 'clc']
score = [model_id, train_data_name, test_data_name] + score + [float(Fpa) , float(CLC)]

In [28]:
heading

['model_id',
 'train_data_name',
 'test_data_name',
 'loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error',
 'fpa',
 'clc']

In [29]:
score

[7,
 'ant-1.3',
 'ant-1.4',
 1.1667767763137817,
 1.1667767763137817,
 0.8208286166191101,
 1.080174446105957,
 0.38576197624206543,
 0.4754561483860016,
 0.48384326696395874]

In [30]:
# Writing the results to csv file.
with open(path_to_save + '_metric_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(heading)
    writer.writerow(score)


In [31]:
# Loading the model and to make sure that the model is saved properly.
model_loaded = load_model(path_to_save)